In [100]:
import collections
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
%matplotlib inline

In [125]:
def generate_abundance_table(initial_profile_file, taxonomic_level):
    taxonomic_levels = collections.OrderedDict()
    taxonomic_levels['kingdom']='k__'
    taxonomic_levels['class'] = 'c__'
    taxonomic_levels['order'] = 'o__'
    taxonomic_levels['family'] = 'f__'
    taxonomic_levels['genus'] = 'g__'
    taxonomic_levels['species'] = 's__'
    taxonomic_levels['strain'] = 't__'

    # Ordered dict manipulation to figure out what the next key is... excessively verbose in an attempt at clarity
    desired_tax_rank = taxonomic_levels[taxonomic_level]
    index_tax_rank = tuple(taxonomic_levels).index(taxonomic_level)
    tax_key_list = list(taxonomic_levels.keys())
    next_tax_level = tax_key_list[index_tax_rank + 1]
    next_tax_rank = taxonomic_levels[next_tax_level]

    output_file = initial_profile_file.replace('_profile','_profile_{}'.format(taxonomic_level))

    p = subprocess.Popen("grep -E '({0})| "
                         "(^ID)' {2} "
                         "| grep -v '{1}' "
                         "| sed 's/^.*{0}//g' > "
                         "{3}".format(desired_tax_rank, next_tax_rank, initial_profile_file, output_file),
                         shell=True,
                         executable='/bin/bash')
    p.wait()
        
def generate_figure(metagenome_id, taxonomic_level):
    df = pd.read_csv('/mnt/scratch/Forest/SRA_carrot_project/metaphlan2/{0}/{0}_profile_{1}.txt'.format(metagenome_id, 
                                                                                                        taxonomic_level),
                    delimiter='\t', names=['genus','proportion'])
    
    cs=cm.spring(np.linspace(0, 100, len(df)))
    
    fig = plt.pie(
        df['proportion'],
        labels=df['genus'],
        shadow=False,
        #colors=cs,
        startangle=90,
        autopct='%1.1f%%',
        )

    # View the plot drop above
    plt.axis('equal')

    # Save figure
    plt.savefig('/mnt/scratch/Forest/SRA_carrot_project/metaphlan2/{0}/{0}_{1}_pie.png'.format(metagenome_id, taxonomic_level), 
                bbox_inches='tight', 
                dpi=200)
    plt.clf()

In [102]:
worklist = ['SRR3747715',
            'SRR3747760',
            'SRR3747811',
            'SRR3747713',
            'SRR3747750',
            'SRR3747761',
            'SRR3747812',
            'SRR3747714',
            'SRR3747751',  
            'SRR3747798',  
            'SRR3747847']

In [126]:
for item in worklist:
    generate_abundance_table(('/mnt/scratch/Forest/SRA_carrot_project/metaphlan2/{0}/{0}_profile.txt'.format(item)),'genus')
    generate_figure(item, 'genus')